<a href="https://colab.research.google.com/github/47Alain/LISP-Interpreter-/blob/main/Copy_of_monte_carlo_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# \~Toss or Hold'em Monte Carlo Practice\~ (Pokerbots 2026)




As we discussed in the previous lecture, Monte Carlo is a method for estimating the unknown using a large random sample of outcomes.

Today, we will be using this to build a smart tosser! Our most recent reference bot simply tosses the smallest ranked card, but what if instead we use Monte Carlo? Before reading on, take some time to try and brainstorm how this could work.

---

Essentially, we want to randomly simulate game endings from the point when it is our turn to toss, and see which card seems best to toss!

First, let's download `pkrbot`, which takes care of a lot of logic for us:

In [1]:
!pip install pkrbot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 641.4/641.4 kB 14.8 MB/s eta 0:00:00


Also we would like to import the necessary libraries (just `pkrbot` and `random`), and set up some testing infrastracture (no need to change anything here)

In [2]:
import pkrbot
import random

def _tests_pkrbotify():
    assert pkrbotify([]) == []
    assert pkrbotify(['Ah', 'As', 'Td', '9c']) == [pkrbot.Card('Ah'), pkrbot.Card('As'), pkrbot.Card('Td'), pkrbot.Card('9c')]
    print("Test cases passed!")

def _tests_get_updated_deck():
    deck = get_updated_deck([pkrbot.Card('5h'), pkrbot.Card('8h')], [pkrbot.Card('9s'), pkrbot.Card('As'), pkrbot.Card('Kc')])
    assert len(deck.cards) == 47
    assert pkrbot.Card('5h') not in deck.cards
    assert pkrbot.Card('8h') not in deck.cards
    assert pkrbot.Card('9s') not in deck.cards
    assert pkrbot.Card('As') not in deck.cards
    assert pkrbot.Card('Kc') not in deck.cards
    print("Test cases passed!")

def _tests_monte_carlo_iteration():
    my_cards, board_cards = pkrbotify(['As', 'Ah', 'Ad']), pkrbotify(['8c', '8s'])
    deck = get_updated_deck(my_cards, board_cards)
    wins = monte_carlo_iteration(deck, my_cards, board_cards)
    assert type(wins) == list
    assert len(wins) == 3
    assert (wins[0] == 0 or wins[0] == 0.5 or wins[0] == 1) and (wins[1] == 0 or wins[1] == 0.5 or wins[1] == 1) and (wins[2] == 0 or wins[2] == 0.5 or wins[2] == 1)
    print("Output format tests passed!")

def _tests_argmax():
    assert argmax([3.0, 5.0]) == 1
    assert argmax([0.5, 0.5, 0]) == 0
    assert argmax([1.0, 2.0, 3.0, 4.0, 2.0, 4.0]) == 3
    assert argmax([0, 0, 0, 0, 0, 0]) == 0
    assert argmax([0, 0, 0, 0, 0, 1]) == 5
    print("Test cases passed!")

def _tests_monte_carlo_toss():
    if MC_ITERATIONS < 100:
        print(f"Note: since MC_ITERATIONS is < 100 ({MC_ITERATIONS}), test cases might have lower chance of passing")
    assert monte_carlo_toss(['Ah', 'Kc', '2d'], ['Ad', 'As']) != 0
    assert monte_carlo_toss(['Ah', '2d', 'Qc'], ['Ac', 'As']) == 1
    assert monte_carlo_toss(['Kh', 'Jc', '6d'], ['Qd', 'Ts', '9s']) == 2
    assert monte_carlo_toss(['Kh', 'Jh', '6d'], ['Qh', 'Th']) == 2
    assert monte_carlo_toss(['Kh', 'Js', '6h'], ['Qh', 'Th', '9h']) == 1
    print("Test cases passed! (does not guarantee correctness, but a correct solution should pass these tests)")

## Strings to Cards

Now, let's implement a function which turns a list of cards represented as strings (i.e. `'Ah'` or `'2s'`) into one of `pkrbot.Card` objects.

>Note: `pkrbot.Card` has a constructor which takes in a string exactly of the given format

In [4]:
def pkrbotify(cards_list):
    return [pkrbot.Card(card_str) for card_str in cards_list]

# Tests (no need to change)
_tests_pkrbotify()

Test cases passed!


## Remaining Deck

We now want to create an object representing the remaining cards in the deck (excluding our cards and the board cards). To do so, we can use the `pkrbot.Deck` object. In its constructor, we can pass `rng_seeded` so that test cases can be deterministic. To remove a card which is in play, we can use `deck.cards.remove(card)` since `deck.cards` is a Python list!

In [7]:
# assume my_cards and board_cards are already pkrbotified
def get_updated_deck(my_cards, board_cards):
    deck = pkrbot.Deck()
    for card in my_cards:
        deck.cards.remove(card)
    for card in board_cards:
        deck.cards.remove(card)
    return deck

# Tests (no need to change)
_tests_get_updated_deck()

Test cases passed!


## Monte Carlo Single Iteration

Here comes the exciting part of the exercise! Actually implementing an iteration of Monte Carlo :)

This is not an easy task, so we included some hints for you to help you out. As for the spec of the function, it will input the remaining deck in `deck`, and the `pkrbotified` cards in `my_cards` and `board_cards`. As for the output, we want to return a list of length 3, each index `i` representing our points this iteration if we toss `my_cards[i]`. We assign winning `1` point, drawing `0.5` points, and losing `0` points (note that this does not have to be the case, it is simply a design choice). Additionally, we will assume that our opponent chooses uniformly at random for any toss action they perform (again, a design choice).

To sample cards from the deck you can use `deck.sample(n)` which will return `n` cards chosen uniformly at random. Also, to evaluate the strength of hands, you can use `pkrbot.evaluate(available_cards)` where `available_cards` is a list of `Card` objects (at least 5, at most 52). The function will return a number, and the greater the number, the better the best 5-card hand from `available_cards`. Good luck!  

<details>
  <summary>Hint 1</summary>
    You might notice that if you sample once for the board and once for the opponent, you might end up with overlapping cards. Make sure to only sample once.
</details>

<details>
  <summary>Hint 2</summary>
    At this state of the game, it is possible for our opponent to have 2 or 3 cards in their hand. But does this affect our logic given that they toss a card chosen uniformly at random from their hand?
</details>

<details>
  <summary>Hint 3</summary>
    With Monte Carlo here, we esentially want to try (for a given sample outcome of the game) tossing each of our cards, and seeing how we perform in each case. The only information that changes with a toss is the card from our original hole cards which ends up in our opponent's 8-card hand.
</details>

<details>
  <summary>Hint 4</summary>
    Don't forget, to concatenate lists easily in Python, you can use the + sign. Also, Python supports splicing within lists (feel free to search up how this works). You can use this to decide which cards out of this iteration's sample you assign to the opponent's hidden cards vs. the cards which end up on the board as community cards.
</details>

If you need help at any point, feel free to ask :)

In [9]:
def monte_carlo_iteration(deck, my_cards, board_cards):
    # Determine how many community cards are still needed
    num_community_cards_to_draw = 5 - len(board_cards)

    # Total cards to sample: 2 for opponent's initial hand + remaining community cards
    num_to_sample = 2 + num_community_cards_to_draw

    # Sample cards from the remaining deck
    sampled_cards = deck.sample(num_to_sample)

    # Distribute sampled cards
    opponent_initial_hand = sampled_cards[:2]
    newly_drawn_community_cards = sampled_cards[2:]
    all_community_cards = board_cards + newly_drawn_community_cards

    results = [0.0] * len(my_cards)

    # Iterate through each card we could potentially toss
    for toss_idx in range(len(my_cards)):
        card_we_toss = my_cards[toss_idx]

        # Our remaining 2 cards after tossing one
        my_active_hand = [card for i, card in enumerate(my_cards) if i != toss_idx]

        # Our full available cards for evaluation (2 active + 5 community)
        my_eval_cards = my_active_hand + all_community_cards

        # Opponent's full available cards for evaluation (2 initial + our 1 tossed + 5 community)
        opponent_eval_cards = opponent_initial_hand + [card_we_toss] + all_community_cards

        # Evaluate both hands
        my_score = pkrbot.evaluate(my_eval_cards)
        opponent_score = pkrbot.evaluate(opponent_eval_cards)

        # Determine outcome
        if my_score > opponent_score:
            results[toss_idx] = 1.0  # Win
        elif my_score < opponent_score:
            results[toss_idx] = 0.0  # Loss
        else:
            results[toss_idx] = 0.5  # Draw

    return results

# Tests just test output format here (no need to change)
_tests_monte_carlo_iteration()

Output format tests passed!


## Argmax

Before writing the final Monte Carlo simulation, implement a simple `argmax` function below (`argmax` is an operation which is done on a list, and it returns the index of the maximum item in the list). In this case, our list will contain floating point numbers. In case of ties, return the lower index. The list is guaranteed not to be empty.

In [10]:
def argmax(my_list):
    # TODO: your code here!
    for i in range(len(my_list)):
        if my_list[i] == max(my_list):
            return i

# Tests (no need to change)
_tests_argmax()

Test cases passed!


## The Real Deal

Yay! We're finally ready to implement our full Monte Carlo simulation using the abstractions we made!

Below, implement a full Monte Carlo simulation which, given `my_cards` (of which there are 3) and the current `board_cards`, decides which card to toss. Return the index of the card (for deciding, we can simply go with the card which has the highest expected win rate).

In [12]:
# a constant we define to determine how many iterations of Monte Carlo we actually run
MC_ITERATIONS = 1000 # feel free to play around with this number, but remember: the Law of Large Numbers says that the higher it is, the more precise our guess
def monte_carlo_toss(my_cards_str, board_cards_str):
    my_cards = pkrbotify(my_cards_str)
    board_cards = pkrbotify(board_cards_str)

    # Initialize total scores for each card we might toss
    total_scores = [0.0] * len(my_cards)

    for _ in range(MC_ITERATIONS):
        deck = get_updated_deck(my_cards, board_cards)
        iteration_scores = monte_carlo_iteration(deck, my_cards, board_cards)

        # Aggregate scores from this iteration
        for i in range(len(my_cards)):
            total_scores[i] += iteration_scores[i]

    # Find the card with the highest average score (expected win rate)
    return argmax(total_scores)

# Test cases (no need to change)
_tests_monte_carlo_toss()

Test cases passed! (does not guarantee correctness, but a correct solution should pass these tests)


## Final remarks

Hopefully now you have a better understanding of Monte Carlo simulation and how we can use it to make decisions about how to act when the future is unknown.

Let us know if you have any questions or need any help at any point.